In [1]:
# 1-Read and show video stream , capture images
#2- Detect Faces and Show bounding box (haarcascade)
#3-Flatten the largest face image(gray scale) and save in a numpy array 
#4-Repeat the above for multiple people to generate training data


In [2]:
import cv2
import numpy as np 
import os

In [3]:
#inlit camera 
cap=cv2.VideoCapture(0)

In [4]:

if not cap.isOpened():
    print("Error: Could not open video capture.")
    exit()

# Load the pre-trained Haar Cascade model for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize variables
face_data = []
skip = 0
dataset_path = "./"  # Define the dataset path
file_name = "face_data"  # Define the file name
face_size = (100, 100)  # Desired face size (width, height)

# Ensure the dataset path exists
os.makedirs(dataset_path, exist_ok=True)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        continue

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
    faces = sorted(faces, key=lambda f: f[2] * f[3])
    
    if len(faces) > 0:  # Select the largest face
        face = faces[-1]
        x, y, w, h = face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)

        offset = 10
        # Ensure the face selection is within bounds
        y_start = max(0, y - offset)
        y_end = min(frame.shape[0], y + h + offset)
        x_start = max(0, x - offset)
        x_end = min(frame.shape[1], x + w + offset)

        face_selection = frame[y_start:y_end, x_start:x_end]
        face_selection = cv2.resize(face_selection, face_size)  # Resize to a consistent shape

        if skip % 10 == 0:
            face_data.append(face_selection)
            print(len(face_data))
        
        cv2.imshow("Face Selection", face_selection)

    # Display the frame with detected faces
    cv2.imshow("Video Frame", frame)

    # Wait for user input - 'q' to stop the loop
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break            
    skip += 1

# Convert our face list array into a numpy array
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0], -1))
print(face_data.shape)

# Save the face data to a numpy file
np.save(os.path.join(dataset_path, file_name + '.npy'), face_data)
print("Data successfully saved at " + os.path.join(dataset_path, file_name + '.npy'))

cap.release()
cv2.destroyAllWindows()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
(21, 30000)
Data successfully saved at ./face_data.npy
